In [1]:
"""將tensorflow檔轉成tensorflow lite，使機器負載量變小"""
from tensorflow import lite
from tensorflow.keras import models

# Parameters
keras_model_filename = './h5/recording23.h5' #訓練好的模型
tflite_filename = './tflite/recording23.tflite' #預建置檔案

# Convert model to TF Lite model
model = models.load_model(keras_model_filename) #載入本來的模型
converter = lite.TFLiteConverter.from_keras_model(model) #將模型載入轉換器
tflite_model = converter.convert() #進行轉換
open(tflite_filename, 'wb').write(tflite_model) #輸出轉換後的模型

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmp8kyshiym\assets


285876

In [1]:
import openpyxl
import pandas as pd
import sounddevice as sd
import numpy as np
import scipy.signal
import python_speech_features
import tensorflow as tf
from PyQt5.QtCore import QTime
import librosa

num_mfcc = 23 #回傳mfcc的量

model_path = './tflite/recording23.tflite'
words = ['ㄏㄧㄡ', 'ㄟ', '吼', '啦', '嗯', '的一個', '的這個', '的那個', '著', '那', '那那個', '阿']#答案對應到的字詞
data = []
start = 0 #開始的索引值
end = 4000 #結束的索引值
s = 0.5 #sec
m = 0 #min
duration = 180 #讀取音檔時間的總和 (3分鐘)
sample_rate = 8000 #取樣率

#載入音檔
f_name = 'C:/Users/user/Desktop/testREC.wav'
y,sr= librosa.load(f_name,sr=sample_rate,duration=duration)
t = y.size / float(sr)
print("y =>",y)
print("y_len =",len(y))
print("Time =",t,"sec\n")

# Sliding window
window = np.zeros(8000)#取樣音頻數據變數

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# This gets called every 0.5 seconds
def sound_file(window,s,m):
    S = np.abs(librosa.stft(window)) #將整個window音訊做stft，並轉成絕對值
    #print("np.sum(S):" + str(np.sum(S)))
    
    if np.sum(S) > 3000: #判斷是否有講話
        # Compute features
        mfccs = python_speech_features.base.mfcc(window, #輸入訊號
                                            samplerate=8000, #取樣率
                                            winlen=0.256, #音框涵蓋時間
                                            winstep=0.050, #音框間距離
                                            numcep=num_mfcc, #返回係數的量
                                            nfilt=26, #過濾器數量
                                            nfft=2048,#FFT大小
                                            preemph=0.0,#不用預強化濾波器
                                            ceplifter=0,#ROBUST
                                            appendEnergy=True,#係數0的話對被替代成總音框能量的對數
                                            winfunc=np.hanning)#hanning window
        mfccs = mfccs.transpose()
        
        # Make prediction from model
        in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
        #設定輸入張量
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        #進行預測
        interpreter.invoke()
        #取得輸出張量
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        val = output_data[0]#取得預測值
        val = val.tolist() #np.ndarray to list
        list_val_max = max(val) #取得最大值
        list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
        
        #if list_val_maxIndex == 0:
            
        if(list_val_maxIndex>=0 and list_val_max >= 0.5):
            data.append(str(m) + "分" + "{:.1f}秒".format(s) + "  出現：" + str(words[list_val_maxIndex]) 
                        + "  預測值：{:.3f}".format(list_val_max)+"\n")             
            
while True:
    window[0:4000] = y[start:end] #把音訊載入window
    start = start + 4000 #向後移動
    end = end + 4000 #向後移動
    window[4000:] = y[start:end] #把音訊載入window
    s = float(s + 0.5) #增加秒數
    if(s == 60): #60 sec <-> 1 min
        s = 0
        m = m + 1
    sound_file(window,s,m) #呼叫sound_file()
    if(end == 1440000): #如果移動到最後break       
        break

for i in range(len(data)):
        print(data[i])
        
print("總共預測：",len(data),"個")

s1 = pd.Series(data)
s1.to_excel('RecData.xlsx')
fn = 'RecData.xlsx'
wb = openpyxl.load_workbook(fn)
ws = wb.active
ws.move_range("B2:G500", rows=-1, cols=-1)
wb.save(fn)

y => [ 5.1879883e-04  3.9672852e-04  2.1362305e-04 ... -7.3852539e-03
 -5.0048828e-03 -3.0517578e-05]
y_len = 1440000
Time = 180.0 sec

0分3.5秒  出現：嗯  預測值：0.628

0分4.0秒  出現：嗯  預測值：0.950

0分4.5秒  出現：的一個  預測值：0.692

0分5.0秒  出現：的一個  預測值：1.000

0分5.5秒  出現：的一個  預測值：1.000

0分6.0秒  出現：嗯  預測值：0.756

0分7.0秒  出現：嗯  預測值：0.931

0分7.5秒  出現：的一個  預測值：0.712

0分8.0秒  出現：嗯  預測值：1.000

0分8.5秒  出現：嗯  預測值：0.957

0分9.0秒  出現：的一個  預測值：0.986

0分9.5秒  出現：嗯  預測值：0.984

0分10.5秒  出現：的一個  預測值：1.000

0分11.0秒  出現：的一個  預測值：0.999

0分11.5秒  出現：嗯  預測值：0.932

0分12.0秒  出現：的一個  預測值：0.845

0分12.5秒  出現：的一個  預測值：1.000

0分13.0秒  出現：的一個  預測值：1.000

0分13.5秒  出現：嗯  預測值：0.999

0分14.0秒  出現：嗯  預測值：0.884

0分14.5秒  出現：的一個  預測值：0.995

0分15.0秒  出現：的一個  預測值：0.537

0分15.5秒  出現：的一個  預測值：0.974

0分16.0秒  出現：嗯  預測值：0.980

0分16.5秒  出現：的一個  預測值：0.913

0分17.0秒  出現：的一個  預測值：0.996

0分17.5秒  出現：嗯  預測值：0.910

0分18.0秒  出現：嗯  預測值：0.701

0分18.5秒  出現：的一個  預測值：0.967

0分19.0秒  出現：的一個  預測值：1.000

0分20.5秒  出現：嗯  預測值：0.999

0分21.0秒  出現：的一個  預測值：1.000

0分21.5秒 